<center> 
<h3><b> Adnan Rashid </b><\h3>
<br><br>
    <nav>
    | <a href="00-Contents-Setup.ipynb"> Home Page </a> | 
    <a href="http://bit.ly/cka_notes_original"> Kubernetes Notes </a> |
    <a href="https://adnan.study"> Website </a> |
    <a href="https://www.instagram.com/adnans_techie_studies/"> Instagram </a> |
    <a href="https://www.linkedin.com/in/adnanrashid1/"> LinkedIn </a> |
    </nav>
</center>

<h1 style="background:black"><code style="background:black;color:white"><center>Deploy Official Kubernetes Dashboard</center></code></h1>

<div class="alert alert-block alert-warning">
<font style="color:black">
<center>In this section we will deploy and access the official Kubernetes dashboard. <br><br>Kubernetes Dashboard is a general purpose, web-based UI for Kubernetes clusters. <br><br>It allows users to manage applications running in the cluster and troubleshoot them, as well as manage the cluster itself. </center>
    </div>
    </font>
    </center>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/5.jpg" width="50%" height="50%" border=5/>

## Pre-requisite

* Check your variables are loaded

In [ ]:
%store

* Load in variable store

In [ ]:
%store -r 

* Load variables into notebook

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env AWS_DEFAULT_REGION=$AWS_DEFAULT_REGION
%env ACCOUNT_ID=$ACCOUNT_ID

* Check identity 

In [ ]:
!aws sts get-caller-identity

<br>

---
---

<br>

* Deploy the dashboard

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.0.0/aio/deploy/recommended.yaml

* In a separate terminal run the following to proxy to the dashboard

<p style="background:black">
<code style="background:black;color:white">> kubectl proxy --port=8080 --address=0.0.0.0 --disable-filter=true &
</code>
</p>



* Browse the following to see the dashboard

In [ ]:
!curl localhost:8080/api/v1/namespaces/kubernetes-dashboard/services/https:kubernetes-dashboard:/proxy/

* Get token for dashboard to access

In [ ]:
!aws eks get-token --cluster-name eksworkshop-eksctl | jq -r '.status.token'

* This is what the dashbord looks like once you enter your token

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/dashboard.png" width="80%" height="80%" border=5/>

* If you find the dashboard interesting you can learn more about it here: https://kubernetes.io/docs/tasks/access-application-cluster/web-ui-dashboard/

## <br><br>Clean Up 

* Stop kubectl proxy

In [ ]:
!pkill -f 'kubectl proxy --port=8080'

* Remove the dashboard

In [ ]:
!kubectl delete -f https://raw.githubusercontent.com/kubernetes/dashboard/v2.0.0/aio/deploy/recommended.yaml

<br><br>
<center>
<h1 style="background:black">
<code style="background:black;color:white">
Deploy Example MicroServices
</code>

<div class="alert alert-block alert-warning">
<font style="color:black">
<center>In this section we deploy a sample application which uses an ELB load balancer. <br><br> We will also scale the backend and frontend of the application.  <br><br>We are currently doing this manually but later will look at how to do it in an automated manner </center>
</div>
</font>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/7.jpg" width="50%" height="50%" border=5/>

* [Deploy NodeJS Backend API](#nodejs)
* [Deploy Crystal Backend API](#crystal)
* [Check Service Types](#types)
* [Ensure the ELB Service Role Exists](#elbservice)
* [Deploy Frontend Service](#frontend)
* [Find the Service Address](#find)
* [Scale the Backend Services](#backend)
* [Scale the Frontend](#frontend)
* [Cleanup](#cleanup)

## <br><br>Deploy NodeJS Backend API <a class="anchor" id="nodejs"></a>

We will deploy 2 services which are 

* <b>deployment.yaml </b>
* <b>service.yaml </b>

This is a deployment which has a single replica and builds a container which the application on port 3000 it specificies the strategy for updates. 

In [ ]:
!cat ecsdemo-nodejs/kubernetes/deployment.yaml

This is a service which specifics the app via labels and the ports

In [ ]:
!cat ecsdemo-nodejs/kubernetes/service.yaml

In [ ]:
!kubectl apply -f ecsdemo-nodejs/kubernetes/deployment.yaml

In [ ]:
!kubectl apply -f ecsdemo-nodejs/kubernetes/service.yaml

* didnt know what the hll was going on so checked pods 

In [ ]:
!kubectl get pods

## <br><br>Deploy Crystal Backend API <a class="anchor" id="crystal"></a>

* Lets bring u the Crystal Backend API 

In [ ]:
!kubectl apply -f ecsdemo-crystal/kubernetes/deployment.yaml

In [ ]:
!kubectl apply -f ecsdemo-crystal/kubernetes/service.yaml

* We can check the progress with the below

In [ ]:
!kubectl get deployment ecsdemo-crystal

## <br><br>Check Service Types<a class="anchor" id="types"></a>

* Before we bring up the frontend service, lets have a look at the service types we are using 

In [ ]:
!cat ecsdemo-frontend/kubernetes/service.yaml

Note type: LoadBalancer

This will configure the ELB to handle incoming traffic to this service

* Compare this to the following file for one of our backend services

In [ ]:
!cat ecsdemo-frontend/kubernetes/service.yaml

<br><br>Notice there is no specific service type described. 

When we check the kubernetes documentation we find that the default type is ClusterIP. This exposes the service on a cluster-internal IP. 

Choosing this value makes the service reachable from within the cluster.

## <br><br>Ensure the ELB Service Role Exists <a class="anchor" id="elbservice"></a>

In some AWS accounts that have never created a load balancer before, its possible that the service role for ELB might not exist yet. 

* We can check the role, if it is missing then create it

In [ ]:
!aws iam get-role --role-name "AWSServiceRoleForElasticLoadBalancing" || aws iam create-service-linked-role --aws-service-name "elasticloadbalancing.amazonaws.com"

## <br><br>Deploy Frontend Service<a class="anchor" id="frontend"></a>

* Lets bring up the Ruby Frontend 

In [ ]:
!kubectl apply -f ecsdemo-frontend/kubernetes/deployment.yaml

In [ ]:
!kubectl apply -f ecsdemo-frontend/kubernetes/ingress.yaml

In [ ]:
!kubectl apply -f ecsdemo-frontend/kubernetes/service.yaml

* Check the deployment status 

In [ ]:
!kubectl get deployment ecsdemo-frontend 

## <br><br>Find the service address<a class="anchor" id="find"></a>

Now that we have a running service that is type: LoadBalancer, we need to find the ELB's address. 

* We can do this by using the 'get services' operation 

In [ ]:
!kubectl get service ecsdemo-frontend

<br><Br>Notice the field isn't wide enough to show the FQDN of the ELB. 

* We can adjust the output format with this command

In [ ]:
!kubectl get service ecsdemo-frontend -o wide

* You will see a website similar to below

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/2.png" width="60%" height="60%" border=5/></center>

## <br><br>Scale the backend services <a class="anchor" id="backend"></a>

We only launched a single container of each

* Check this running the below 

In [ ]:
!kubectl get deployments

* Now let's scale up the backend services 

In [ ]:
!kubectl scale deployment ecsdemo-nodejs --replicas=3

In [ ]:
!kubectl scale deployment ecsdemo-crystal --replicas=3

* Check the deployment again 

In [ ]:
!kubectl get deployments

## <br><br>Scale the Frontend Services<a class="anchor" id="frontend"></a>

* Lets also scale our frontend services

In [ ]:
!kubectl get deployments

In [ ]:
!kubectl scale deployment ecsdemo-frontend --replicas=3

In [ ]:
!kubectl get deployments

* At this point you will also be able to see traffic now flowing through the ELB 

## <br><br>Cleanup<a class="anchor" id="cleanup"></a>

Let's clean it up because we will automate this deployment with Helm next 

This can take a few minutes

In [ ]:
!kubectl delete -f ecsdemo-frontend/kubernetes/service.yaml
!kubectl delete -f ecsdemo-frontend/kubernetes/deployment.yaml

In [ ]:
!kubectl delete -f ecsdemo-crystal/kubernetes/service.yaml
!kubectl delete -f ecsdemo-crystal/kubernetes/deployment.yaml

In [ ]:
!kubectl delete -f ecsdemo-nodejs/kubernetes/service.yaml
!kubectl delete -f ecsdemo-nodejs/kubernetes/deployment.yaml

In [ ]:
!kubectl delete ingress ecsdemo-frontend